In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm import tqdm

c:\Install\anaconda\envs\wdzd_project\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Install\anaconda\envs\wdzd_project\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
df_raw = pd.read_csv('data\original.csv', encoding='windows-1252', header=None, usecols=[2, 5])

In [3]:
df_raw['date'] = pd.to_datetime(df_raw[2].astype(str))

C:\Users\Przemek\AppData\Local\Temp\ipykernel_5212\146429819.py:1: FutureWarning: Parsed string "Mon Apr 06 22:19:45 PDT 2009" included an un-recognized timezone "PDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df_raw['date'] = pd.to_datetime(df_raw[2].astype(str))


In [ ]:
df = df_raw.drop(columns=[2]).rename(columns={5: 'tweet'})

In [13]:
df.to_parquet('data\\refined.parquet')
df.dtypes

tweet            object
date     datetime64[ns]
dtype: object

In [5]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# Setup pipeline on GPU
fe_pipeline = pipeline(
    task="feature-extraction",
    model=model,
    torch_dtype=torch.float16,
    device='cuda:0',
    tokenizer=tokenizer
)

c:\Install\anaconda\envs\wdzd_project\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Install\anaconda\envs\wdzd_project\lib\site-packages\transformers\modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted 

In [6]:
from datasets import Dataset
ds = Dataset.from_pandas(df[['tweet']])
batch_size=32
texts = df['tweet'].tolist()
output_file = "D:\\WDZD\\bertweet_embeddings.pt"

examples_count = 1000# len(texts)

with open(output_file, "wb") as f:
    for i in tqdm(range(0, examples_count, batch_size)):
        batch = texts[i:i + batch_size]
        # Inference
        with torch.no_grad():
            outputs_list = fe_pipeline(batch, padding=True, truncation=True, return_tensors="pt")
            outputs = torch.stack([emb[0, 0, :] for emb in outputs_list])

        emb = torch.tensor(outputs).float().cpu().numpy() # 32 x 768
        f.write(emb.tobytes())  # write raw bytes


  0%|          | 0/32 [00:00<?, ?it/s]C:\Users\Przemek\AppData\Local\Temp\ipykernel_5212\738702710.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(outputs).float().cpu().numpy() # 32 x 768
 31%|███▏      | 10/32 [00:05<00:08,  2.50it/s]c:\Install\anaconda\envs\wdzd_project\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 32/32 [00:13<00:00,  2.34it/s]


In [7]:
import numpy as np
with open(output_file, "rb") as f:
    # header = np.frombuffer(f.read(8), dtype=np.int32)
    # total_embeddings, embedding_dim = header
    data = f.read()
    embeddings = np.frombuffer(data, dtype=np.float32).reshape(int(np.ceil(examples_count / 32) * 32), 768)

In [14]:
embeddings.shape

(1024, 768)

In [ ]:
df_sorted = df.sort_values(by='date')
df['time_group'] = pd.qcut(df_sorted['date'].rank(method='first'), q=100, labels=False)
df

In [ ]:
df = df.iloc[:int(np.ceil(examples_count / 32) * 32)]

In [ ]:
time_grouped_embeddings = {}
for group in range(100):
    indices = df[df['time_group'] == group].index
    time_grouped_embeddings[group] = embeddings[indices]

In [ ]:
len(time_grouped_embeddings)

In [ ]:
time_grouped_embeddings[0].shape

In [ ]:
import umap
import numpy as np
import joblib

# Assuming your embeddings are in this variable
# embeddings.shape == (1600000, 768)
umaps = dict()
for key in time_grouped_embeddings:
    if time_grouped_embeddings[key].shape[0] == 0:
        continue
    umap_model = umap.UMAP(
        n_neighbors=5,        # controls local vs global structure
        n_components=2,        # 2D for visualization; use 3 or more if needed
        metric='cosine',       # or 'euclidean'
        n_jobs=-1
    )
    
    embedding_2d = umap_model.fit_transform(time_grouped_embeddings[key])
    joblib.dump(umap_model, f"models/umap_model_{key}.pkl")
    print("UMAP output shape:", embedding_2d.shape)

In [ ]:
umap0 = joblib.load(f"models/umap_model_{0}.pkl")

In [ ]:
len(fe_pipeline("I like trains")[0][0])

In [ ]:
embedding = torch.tensor(fe_pipeline("I like trains")[0][0])

In [ ]:
umap0.transform(embedding.reshape(1, -1))